In [2]:
'''
IMPORTS
'''

import numpy as np 
import pandas as pd
from sklearn.utils import shuffle
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [3]:
'''
PREPROCESSING
'''

M = fetch_openml(data_id=24)
x, y = M.data, M.target

#! Convert the data to a pandas DataFrame
M_features_df = pd.DataFrame(x, columns=M.feature_names)
M_targets_df = pd.DataFrame(y, columns=['class'])

M_everything_df = pd.concat([M_features_df, M_targets_df], axis=1)
# print(M_everything_df.head())

'''
CONVERT TO NUMERICAL DATA
'''
mapping = {
    'cap-shape': {'b': 0, 'c': 1, 'x': 2, 'f': 3, 'k': 4, 's': 5},
    'cap-surface': {'f': 0, 'g': 1, 'y': 2, 's': 3},
    'cap-color': {'n': 0, 'b': 1, 'c': 2, 'g': 3, 'r': 4, 'p': 5, 'u': 6, 'e': 7, 'w': 8, 'y': 9},
    'bruises%3F': {'f': 0, 't': 1},
    'odor': {'a': 0, 'l': 1, 'c': 2, 'y': 3, 'f': 4, 'm': 5, 'n': 6, 'p': 7, 's': 8},
    'gill-attachment': {'a': 0, 'd': 1, 'f': 2, 'n': 3},
    'gill-spacing': {'c': 0, 'w': 1, 'd': 2},
    'gill-size': {'b': 0, 'n': 1},
    'gill-color': {'k': 0, 'n': 1, 'b': 2, 'h': 3, 'g': 4, 'r': 5, 'o': 6, 'p': 7, 'u': 8, 'e': 9, 'w': 10, 'y': 11},
    'stalk-shape': {'e': 0, 't': 1},
    'stalk-root': {'b': 0, 'c': 1, 'u': 2, 'e': 3, 'z': 4, 'r': 5, '?': 6},
    'stalk-surface-above-ring': {'f': 0, 'y': 1, 'k': 2, 's': 3},
    'stalk-surface-below-ring': {'f': 0, 'y': 1, 'k': 2, 's': 3},
    'stalk-color-above-ring': {'n': 0, 'b': 1, 'c': 2, 'g': 3, 'o': 4, 'p': 5, 'e': 6, 'w': 7, 'y': 8},
    'stalk-color-below-ring': {'n': 0, 'b': 1, 'c': 2, 'g': 3, 'o': 4, 'p': 5, 'e': 6, 'w': 7, 'y': 8},
    'veil-type': {'p': 0, 'u': 1},
    'veil-color': {'n': 0, 'o': 1, 'w': 2, 'y': 3},
    'ring-number': {'n': 0, 'o': 1, 't': 2},
    'ring-type': {'c': 0, 'e': 1, 'f': 2, 'l': 3, 'n': 4, 'p': 5, 's': 6, 'z': 7},
    'spore-print-color': {'k': 0, 'n': 1, 'b': 2, 'h': 3, 'r': 4, 'o': 5, 'u': 6, 'w': 7, 'y': 8},
    'population': {'a': 0, 'c': 1, 'n': 2, 's': 3, 'v': 4, 'y': 5},
    'habitat': {'g': 0, 'l': 1, 'm': 2, 'p': 3, 'u': 4, 'w': 5, 'd': 6},
    'class': {'e': 0, 'p': 1}
}
for column, mp in mapping.items():
    M_everything_df[column] = M_everything_df[column].replace(mp)
# print(M_everything_df.head())

le = LabelEncoder()
M_everything_df_encoded = M_everything_df.apply(le.fit_transform)

'''
CLEAN DATA; VERIFY WITH STANDARD DEVIATION AND MEAN
'''

# veil-type has std = mean = 0 
# remove veil-type column
M_everything_df_encoded = M_everything_df_encoded.drop(columns='veil-type')

# change to 1 to see std/mean outputs
if(0):
    print("standard deviations:")
    std = M_everything_df_encoded.std()
    print(std)
    print()
    print("mean:")
    mean = M_everything_df_encoded.mean()
    print(mean)

'''
SPLIT DATA
'''
# Now split and use the encoded DataFrame
train, test = train_test_split(M_everything_df_encoded, test_size=.2, random_state=41)

X_train = train.iloc[:,:-1].values
Y_train = train.iloc[:,-1].values

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values

In [4]:
'''
KNN FUNCTIONS 
'''

# Feature scaling functions

def normalize(X):
    x1_min = min(X_train[:,0])
    x1_max = max(X_train[:,0])
    
    f = lambda x: (x - x1_min)/(x1_max - x1_min)
    X[:,0] = f(X[:,0])

    x2_min = min(X_train[:,1])
    x2_max = max(X_train[:,1])
    
    f = lambda x: (x - x2_min)/(x2_max - x2_min)
    X[:,1] = f(X[:,1])
    
    return X

X = normalize(X_train)

print(X[0:5])

# KNN
def find_neighbors(k, X_tr, new_point):
    neighbor_arr = []
    for i in range(len(X_tr)):
        dist = np.sqrt(sum(np.square(X_tr[i]-new_point)))
        neighbor_arr.append([i, dist])
    neighbor_arr = sorted(neighbor_arr, key = lambda x : x[1])
    
    return neighbor_arr[0:k]

from collections import Counter
def classifier(neighbor_arr):
    class_arr = [Y_train[i[0]] for i in neighbor_arr]
    return Counter(class_arr).most_common(1)[0][0]

new_points = np.array([[-10, -10],
                      [0, 10],
                      [-15, 10],
                      [5, -2]])

new_points = normalize(new_points)
knn = find_neighbors(4, X, new_points[1])
classifier(knn)

[[ 0  0  3  1  6  1  0  0  7  1  0  3  3  3  7  2  1  4  1  5  6]
 [ 0  1  8  1  4  1  0  0 10  1  0  3  0  7  7  2  1  4  3  4  0]
 [ 0  1  8  1  4  1  0  0 10  1  0  0  0  7  7  2  1  4  3  3  4]
 [ 0  1  5  0  2  1  0  1  8  0  0  3  3  7  7  2  1  4  1  4  6]
 [ 0  1  7  0  8  1  0  1  2  1  4  3  3  5  7  2  1  0  7  4  3]]


ValueError: operands could not be broadcast together with shapes (21,) (2,) 

In [ ]:
'''
ANALYZE DATA
'''

from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred))

from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(Y_test, Y_pred))